In [1]:
import google.generativeai as genai
import pandas as pd
import time

In [ ]:
# Necesitaras crear un archivo GEMINI_API_KEYS.txt con las claves de Gemini que tengas
with open('GEMINI_API_KEY.txt') as f:
    # Importante poner .strip() para quitar posibles '\n'
    genai.configure(api_key=f.readline().strip())

In [ ]:
df = pd.read_excel('DESCRIPCIONES_TODAS.xlsx')
df = df.dropna(subset='Código de la especie (id taxon)')
df['Código de la especie (id taxon)'] = df['Código de la especie (id taxon)'].astype(int)
df = df[df.duplicated(subset='Código de la especie (id taxon)', keep=False)]
df

,Nombres comunes,Nombre aceptado,Fuente,Grupo taxonómico,Descripción,Código de la especie (id taxon)
0,Abejaruco europeo,Merops apiaster,SEO BirdLife / VertebradosIbericos - Aves,Aves,"Esta ave, del tamaño de un zorzal común, se di...",10738
1,Abejero europeo,Pernis apivorus,SEO BirdLife / VertebradosIbericos - Aves,Aves,La coloración de esta especie presenta conside...,10739
2,Abubilla común,Upupa epops,SEO BirdLife / VertebradosIbericos - Aves,Aves,La abubilla común es un ave de aspecto inconfu...,11211
3,Agateador europeo,Certhia brachydactyla,SEO BirdLife / VertebradosIbericos - Aves,Aves,"Ave diminuta, insectívora y forestal (dibujo 1...",10733
4,Águila calzada,Hieraaetus pennatus,SEO BirdLife / VertebradosIbericos - Aves,Aves,"Es una rapaz de tamaño medio, formas estilizad...",11959
...,...,...,...,...,...,...
304,"Visón americano (Castellano), Bisoi amerikarra...","Neovison vison (Schreber, 1777)",IEPNB / VertebradosIbericos - Mamíferos,Mamíferos,"Es un mustélido de tamaño medio, de piel lustr...",12409
305,"Calderón Gris (Castellano), Delfín gris (Caste...","Grampus griseus (G. Cuvier, 1812)",IEPNB / VertebradosIbericos - Mamíferos,Mamíferos,Calderón de tamaño más bien grande que se reco...,20075
306,"Orca (Castellano), Espadarte (Castellano), Ork...","Orcinus orca (Linnaeus, 1758)",IEPNB / VertebradosIbericos - Mamíferos,Mamíferos,"La orca es el delfínido de mayor tamaño, llega...",20079
307,"Ballenato de Cuvier (Castellano), Zifio de Cuv...","Ziphius cavirostris G. Cuvier, 1823",IEPNB / VertebradosIbericos - Mamíferos,Mamíferos,Cetáceo de tamaño medio que puede alcanzar los...,20099


In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
df['Descripción Gemini'] = ''
for i, row in df.iterrows():
    if (i+1) % 15 == 0:
        print(f'Starting prompt {i+1}...', end='\r')
        time.sleep(60)
    messages = [
        {'role': 'user',
        'parts': [f"""Te voy a enviar múltiples descripciones de la misma especie, separadas por caracteres de final de línea (\n).
                  Quiero que las combines en una sola, de forma que se mantenga el tono y el formato de una de ellas, eliminando la información redundante.
                  Específicamente, quiero que la descripción resultante sea igual que una de las que yo te incluyo, pero con la información del resto de descripciones adaptadas al mismo tono.
                  Para ello, te recomiendo que partas de la descripción más larga.
                  
                  Estas son las descripciones:]
                  
                  {row['Descripción']}"""]}
    ]
    response = model.generate_content(messages)
    df.loc[i, 'Descripción Gemini'] = response.candidates[0].content.parts[0].text.strip()
print(f'Finished all {len(df)} prompts')
df.rename(columns={'Descripción': 'Descripciones'}).to_excel(f'Descripciones_Combinadas.xlsx', index=False)